<a href="https://www.kaggle.com/code/ryancardwell/orcaswordv1?scriptVersionId=272233829" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#Cell 1
"""
ARC PRIZE 2025 - ORCASWORD SOLVER v7.1 (MCAS + FUNCTIONAL TIME SINKS)
================================================================================
FIXED: Replaced non-functional probabilistic placeholders in costly strategies
(MPA, Variational) with their actual, functional, computationally expensive
logic (O(N!) search, O(|lambda|) search). This restores the necessary time sinks
and maximizes the potential for finding solutions.

Key Architecture: Meta-Cognitive Adaptive System (MCAS)
- Learns optimal strategy order based on task tier and success history.
- Dual Agents (Alpha/Omega) exploit the dynamic ordering and meta-awareness.
"""

import json
import numpy as np
import time 
from typing import List, Dict, Tuple, Optional, Set, Callable
from dataclasses import dataclass, field
from collections import defaultdict
from itertools import permutations
import copy
from math import sqrt, ceil, floor

# =============================================================================
# --- Global Configuration & Time Management (SDP/NSM Parameters) --- (x4 Segments)
# =============================================================================

MAX_PERMUTATION_LENGTH = 5      # Search depth for Multi-Primitive Agent (5! = 120 checks)
MAX_VP_SEARCH_ITERATIONS = 5    # Max lambda search for Variational Primitives
MAX_EIGEN_SOLVE_DIM = 200       # Safety cap for N^3 operations
COST_WEIGHT_FACTOR = 1000       # Multiplier for slow strategies to simulate time consumption

# =============================================================================
# --- CORE DATA STRUCTURES & TASK TIERING --- (x6 Segments)
# =============================================================================

Grid = List[List[int]]
TaskTier = str 

@dataclass
class ARCTask:
    task_id: str
    train_examples: List[Tuple[Grid, Grid]]
    test_inputs: List[Grid]

@dataclass
class StrategyMetrics:
    name: str
    cost: int
    success_count_by_tier: Dict[TaskTier, int] = field(default_factory=lambda: defaultdict(int))
    total_attempts_by_tier: Dict[TaskTier, int] = field(default_factory=lambda: defaultdict(int))
    last_success_time: float = 0.0

@dataclass
class TaskFeatures:
    grid_size_score: int
    unique_colors: int
    density_ratio: float
    symmetry_score: float
    tier: TaskTier = "Medium"

# =============================================================================
# --- ENHANCED GRID OPERATIONS & FEATURE EXTRACTION --- (x20 Segments)
# =============================================================================

class GridOps:
    @staticmethod
    def to_numpy(grid: Grid) -> np.ndarray: return np.array(grid, dtype=np.int32)
    @staticmethod
    def from_numpy(arr: np.ndarray) -> Grid: return arr.tolist()
    @staticmethod
    def grids_equal(g1: Grid, g2: Grid) -> bool:
        if not g1 or not g2 or len(g1) != len(g2) or len(g1[0]) != len(g2[0]): return False
        return all(r1 == r2 for r1, r2 in zip(g1, g2))
    @staticmethod
    def rotate_90(grid: Grid, k: int = 1) -> Grid: return GridOps.from_numpy(np.rot90(GridOps.to_numpy(grid), -k))
    @staticmethod
    def replace_color(grid: Grid, old_color: int, new_color: int) -> Grid:
        return [[new_color if cell == old_color else cell for cell in row] for row in grid]
    
    # [Topological Primitives]
    @staticmethod
    def boundary_extract(grid: Grid) -> Grid:
        I = GridOps.to_numpy(grid); H, W = I.shape; O = np.zeros_like(I)
        for r in range(H):
            for c in range(W):
                if I[r, c] != 0:
                    is_boundary = False
                    for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                        r2, c2 = r + dr, c + dc
                        if not (0 <= r2 < H and 0 <= c2 < W and I[r2, c2] == I[r, c]):
                            is_boundary = True
                            break
                    if is_boundary: O[r, c] = I[r, c]
        return GridOps.from_numpy(O)

    @staticmethod
    def erode_morphological(grid: Grid, iterations: int = 1) -> Grid:
        I = GridOps.to_numpy(grid)
        for _ in range(iterations):
            new_I = I.copy(); H, W = I.shape
            for r in range(1, H - 1):
                for c in range(1, W - 1):
                    if I[r, c] != 0 and np.any(I[r-1:r+2, c-1:c+2] == 0):
                        new_I[r, c] = 0
            I = new_I
        return GridOps.from_numpy(I)

    # [Geometric Primitives]
    @staticmethod
    def invert_colors(grid: Grid) -> Grid:
        I = GridOps.to_numpy(grid); O = np.where(I != 0, 9 - I, 0); return GridOps.from_numpy(O)
    
    @staticmethod
    def center_mass_align(grid: Grid) -> Grid:
        I = GridOps.to_numpy(grid); H, W = I.shape; non_zero = np.argwhere(I != 0)
        if non_zero.size == 0: return grid
        center_r, center_c = non_zero.mean(axis=0)
        target_r, target_c = (H - 1) / 2, (W - 1) / 2
        dy, dx = int(round(target_r - center_r)), int(round(target_c - center_c))
        O = np.roll(np.roll(I, dy, axis=0), dx, axis=1)
        if dy > 0: O[:dy, :] = 0;
        if dy < 0: O[H+dy:, :] = 0
        if dx > 0: O[:, :dx] = 0
        if dx < 0: O[:, W+dx:] = 0
        return GridOps.from_numpy(O)
    
    @staticmethod
    def reflection_map(grid: Grid, axis: str) -> Grid:
        I = GridOps.to_numpy(grid)
        if axis == 'H': return GridOps.from_numpy(I[:, ::-1])
        if axis == 'V': return GridOps.from_numpy(I[::-1, :])
        return grid

    # [Statistical Primitives]
    @staticmethod
    def get_color_mapping(inp: Grid, out: Grid) -> Optional[Dict[int, int]]:
        I_arr, O_arr = GridOps.to_numpy(inp), GridOps.to_numpy(out)
        unique_colors = np.unique(I_arr[I_arr != 0]); color_map = {}
        for c in unique_colors:
            output_colors = np.unique(O_arr[I_arr == c])
            if len(output_colors) == 1:
                color_map[int(c)] = int(output_colors[0])
            elif len(output_colors) > 1: return None 
        if len(color_map.keys()) != len(set(color_map.values())): return None
        return color_map

    # [Feature Extraction Methods for Tiering]
    @staticmethod
    def calculate_features(task: ARCTask) -> TaskFeatures:
        all_inputs = [GridOps.to_numpy(inp) for inp, _ in task.train_examples]
        H, W = len(task.train_examples[0][0]), len(task.train_examples[0][0][0])
        total_cells = H * W * len(task.train_examples)

        unique_colors = 0
        for arr in all_inputs: unique_colors = max(unique_colors, len(np.unique(arr[arr != 0])))
        
        non_zero_count = sum(np.sum(arr != 0) for arr in all_inputs)
        density_ratio = non_zero_count / total_cells if total_cells > 0 else 0.0

        grid_size_score = H * W

        sym_score = 0.0
        for arr in all_inputs:
            h_flip = arr[:, ::-1]
            v_flip = arr[::-1, :]
            sym_score += np.mean(arr == h_flip) + np.mean(arr == v_flip)
        symmetry_score = sym_score / (len(all_inputs) * 2)

        features = TaskFeatures(grid_size_score, unique_colors, density_ratio, symmetry_score)
        features.tier = GridOps.determine_tier(features)
        return features
    
    @staticmethod
    def determine_tier(features: TaskFeatures) -> TaskTier:
        # Simple heuristic mapping to Tiers
        complexity_score = (features.unique_colors * 5) + (features.grid_size_score / 10) + (1 - features.symmetry_score) * 20
        
        if complexity_score < 10: return "Easier"
        elif complexity_score < 25: return "Medium"
        elif complexity_score < 40: return "Hard"
        else: return "Elite"
    
# =============================================================================
# --- TRANSFORMATION STRATEGY HIERARCHY (TIERED COST) --- (x15 Segments)
# =============================================================================

class TransformationStrategy:
    def __init__(self, name: str, cost: int):
        self.name = name
        self.cost = cost 
    
    def can_apply(self, train_examples: List[Tuple[Grid, Grid]]) -> Tuple[bool, Optional[Dict]]:
        return False, None
    
    def apply(self, test_input: Grid, rule_params: Dict) -> Optional[Grid]:
        return test_input

# --- COST 1 PRIMITIVES (TIER: EASIER) ---
class IdentityStrategy(TransformationStrategy):
    def __init__(self): super().__init__("identity", 1)
    def can_apply(self, train_examples): 
        if all(GridOps.grids_equal(inp, out) for inp, out in train_examples): return True, {}
        return False, None
    def apply(self, test_input, rule_params): return test_input

class RotationStrategy(TransformationStrategy):
    def __init__(self, k: int): super().__init__(f"rotate_{k*90}", 1)
    def can_apply(self, train_examples): 
        if all(GridOps.grids_equal(GridOps.rotate_90(inp, self.k), out) for inp, out in train_examples):
            return True, {'k': self.k}
        return False, None
    def apply(self, test_input, rule_params): return GridOps.rotate_90(test_input, rule_params['k'])

class ColorMapStrategy(TransformationStrategy):
    def __init__(self): super().__init__("color_map", 1)
    def can_apply(self, train_examples):
        maps = [GridOps.get_color_mapping(inp, out) for inp, out in train_examples]
        if all(m is not None for m in maps) and all(m == maps[0] for m in maps):
            return True, {'map': maps[0]}
        return False, None
    def apply(self, test_input, rule_params):
        if 'map' in rule_params:
            grid = test_input
            for old, new in rule_params['map'].items():
                grid = GridOps.replace_color(grid, old, new)
            return grid
        return test_input
        
# --- COST 2 PRIMITIVES (TIER: MEDIUM) ---
class CenterMassAlignStrategy(TransformationStrategy):
    def __init__(self): super().__init__("center_mass_align", 2)
    def can_apply(self, train_examples):
        if all(GridOps.grids_equal(GridOps.center_mass_align(inp), out) for inp, out in train_examples): return True, {}
        return False, None
    def apply(self, test_input, rule_params): return GridOps.center_mass_align(test_input)

class BoundaryExtractStrategy(TransformationStrategy):
    def __init__(self): super().__init__("boundary_extract", 2)
    def can_apply(self, train_examples):
        if all(GridOps.grids_equal(GridOps.boundary_extract(inp), out) for inp, out in train_examples): return True, {}
        return False, None
    def apply(self, test_input, rule_params): return GridOps.boundary_extract(test_input)

# --- COST 3 PRIMITIVES (TIER: ELITE/HARD) ---

# Variational Primitive (VP) Base Class (Functional Time Sink: O(|lambda|))
class VariationalPrimitive(TransformationStrategy):
    def __init__(self, name: str, cost: int, op_func: Callable):
        super().__init__(name, cost)
        self.op_func = op_func
        self.lambda_range = list(range(1, MAX_VP_SEARCH_ITERATIONS + 1)) 

    def _fitness(self, transformed_grid: Grid, output_grid: Grid) -> float:
        I_arr, O_arr = GridOps.to_numpy(transformed_grid), GridOps.to_numpy(output_grid)
        return np.sum((I_arr - O_arr)**2) / I_arr.size

    def can_apply(self, train_examples):
        # Time sink: internal SDP search (Iterating over self.lambda_range)
        optimal_lambda = None
        for inp, out in train_examples:
            best_T, best_fit = None, float('inf')
            
            for T in self.lambda_range: 
                transformed_grid = self.op_func(inp, T)
                fit = self._fitness(transformed_grid, out)
                
                if fit < best_fit:
                    best_fit = fit
                    best_T = T
            
            if best_fit > 0.01: return False, None 

            if optimal_lambda is None: optimal_lambda = best_T
            elif optimal_lambda != best_T: return False, None
                
        return True, {'lambda': optimal_lambda}
    
    def apply(self, test_input, rule_params):
        if 'lambda' in rule_params:
            # Need to pass the lambda value for the erosion function
            return self.op_func(test_input, rule_params['lambda']) 
        return test_input

def variational_erode_op(grid: Grid, T: int) -> Grid:
    return GridOps.erode_morphological(grid, iterations=T)

class VariationalErosionStrategy(VariationalPrimitive):
    def __init__(self):
        super().__init__("variational_erosion", 3, variational_erode_op)
        self.lambda_range = list(range(1, 4)) # Limited search space for efficiency

# Multi-Primitive Agent (MPA) (Functional Time Sink: O(N!))
class MultiPrimitiveAgent(TransformationStrategy):
    def __init__(self): 
        super().__init__("multi_primitive_agent", 3)
        # Full set of candidate primitives for the N! search
        self.primitives_pool = [
            GridOps.rotate_90,
            lambda g: GridOps.rotate_90(g, 2), # Rotate 180
            GridOps.boundary_extract,
            GridOps.invert_colors,
            GridOps.center_mass_align
        ]
        self.best_sequence = None

    def can_apply(self, train_examples):
        N = MAX_PERMUTATION_LENGTH
        self.best_sequence = None # Reset sequence for each call
        
        # Core Functional Time Sink: O(N!) search over permutations
        for seq_tuple in permutations(self.primitives_pool, N):
            sequence = list(seq_tuple)
            sequence_matches = True
            
            for inp, out in train_examples:
                current_grid = copy.deepcopy(inp)
                for primitive_func in sequence:
                    current_grid = primitive_func(current_grid)
                
                if not GridOps.grids_equal(current_grid, out):
                    sequence_matches = False
                    break
            
            if sequence_matches:
                self.best_sequence = sequence
                seq_names = [f.__name__ if hasattr(f, '__name__') else "lambda" for f in sequence]
                return True, {'sequence': seq_names}

        return False, None
    
    def apply(self, test_input, rule_params):
        if self.best_sequence:
            current_grid = copy.deepcopy(test_input)
            for primitive_func in self.best_sequence:
                current_grid = primitive_func(current_grid)
            return current_grid
        return test_input

# Placeholder Strategies (Cost 2 & 3 - for completeness and future implementation)
class ExhaustiveToroidalShift(TransformationStrategy):
    def __init__(self): super().__init__("exhaustive_toroidal_shift", 2)
    def can_apply(self, train_examples): return False, {} # Not implemented
    def apply(self, test_input, rule_params): return test_input
class PermutationColorStrategy(TransformationStrategy):
    def __init__(self): super().__init__("permutation_color_map", 2)
    def can_apply(self, train_examples): return False, {} # Not implemented
    def apply(self, test_input, rule_params): return test_input
class LaplacianEigenmapStrategy(TransformationStrategy):
    def __init__(self): super().__init__("laplacian_eigenmap", 3)
    def can_apply(self, train_examples): return False, {} # Not implemented
    def apply(self, test_input, rule_params): return test_input
class DFTMatchStrategy(TransformationStrategy):
    def __init__(self): super().__init__("dft_pattern_match", 3)
    def can_apply(self, train_examples): return False, {} # Not implemented
    def apply(self, test_input, rule_params): return test_input
class SubGridWindowPermutationStrategy(TransformationStrategy):
    def __init__(self): super().__init__("subgrid_window_perm", 3)
    def can_apply(self, train_examples): return False, {} # Not implemented
    def apply(self, test_input, rule_params): return test_input

# =============================================================================
# --- META-LEARNER COMPONENT (ADAPTIVE STRATEGY SELECTION) --- (x10 Segments)
# =============================================================================

class MetaLearner:
    """Stores performance history and determines dynamic strategy order."""
    def __init__(self, all_strategies: List[TransformationStrategy]):
        self.all_strategies = all_strategies
        self.metrics: Dict[str, StrategyMetrics] = {
            s.name: StrategyMetrics(s.name, s.cost) for s in all_strategies
        }
        self.global_solved_task_ids = set()
        print(f"   MetaLearner initialized with {len(all_strategies)} strategies.")

    def update_metrics(self, strategy_name: str, task_tier: TaskTier, success: bool):
        metrics = self.metrics[strategy_name]
        metrics.total_attempts_by_tier[task_tier] += 1
        if success:
            metrics.success_count_by_tier[task_tier] += 1
            metrics.last_success_time = time.time()
            
    def get_success_rate(self, strategy_name: str, task_tier: TaskTier) -> float:
        metrics = self.metrics[strategy_name]
        attempts = metrics.total_attempts_by_tier[task_tier]
        successes = metrics.success_count_by_tier[task_tier]
        # Return -1 if not attempted to ensure attempted strategies are prioritized
        return successes / attempts if attempts > 0 else -1.0 

    def get_dynamic_order(self, task_tier: TaskTier, ideology: str) -> List[TransformationStrategy]:
        """Dynamically sorts strategies based on task tier performance."""
        
        def dynamic_sort_key(strategy: TransformationStrategy) -> Tuple[int, float]:
            success_rate = self.get_success_rate(strategy.name, task_tier)
            cost_factor = strategy.cost
            
            if ideology == 'alpha':
                # Alpha (Fast/Consolidated): Prioritize Low Cost (quick win), then High Success Rate
                # Key: (Cost, -Success Rate) -> Ascending Cost, Descending Success
                return (cost_factor, -success_rate)
            else: # Omega (Complex/Breakthrough): Prioritize High Success Rate, then Low Cost
                # Key: (-Success Rate, Cost) -> Descending Success, Ascending Cost
                return (-success_rate, cost_factor) 

        sorted_strategies = sorted(self.all_strategies, key=dynamic_sort_key)
        
        # Elite methodology check: When Omega is solving Hard/Elite tasks, force high-cost strategies to the front
        if task_tier in ['Hard', 'Elite'] and ideology == 'omega':
            sorted_strategies.sort(key=lambda s: s.cost, reverse=True)
            
        return sorted_strategies

# =============================================================================
# --- ENHANCED SOLVER CORE (MCAS DUAL-AGENT) --- (x15 Segments)
# =============================================================================

class EnhancedARCSolver:
    """Solver supporting Alpha (Fast) and Omega (Complex) ideologies, guided by MetaLearner."""
    
    def __init__(self, ideology: str = 'alpha', meta_learner: Optional[MetaLearner] = None):
        self.ideology = ideology
        self.meta_learner = meta_learner
        
        # Initialize strategies with the full list
        self.strategies = self._get_all_strategies()
        self.stats = {'total': 0, 'solved': 0, 'strategy_success': defaultdict(int)}

    def _get_all_strategies(self) -> List[TransformationStrategy]:
        # Full list of strategies
        return [
            IdentityStrategy(), RotationStrategy(1), RotationStrategy(2), RotationStrategy(3), ColorMapStrategy(), 
            CenterMassAlignStrategy(), BoundaryExtractStrategy(), 
            ExhaustiveToroidalShift(), PermutationColorStrategy(), # Cost 2 Placeholders
            MultiPrimitiveAgent(), VariationalErosionStrategy(), # Functional Time Sinks (Cost 3)
            LaplacianEigenmapStrategy(), DFTMatchStrategy(), SubGridWindowPermutationStrategy() # Cost 3 Placeholders
        ]

    def solve_task(self, task: ARCTask) -> List[Grid]:
        # 1. Task Tiering (Task Analyzer)
        features = GridOps.calculate_features(task)
        task_tier = features.tier
        
        # 2. Dynamic Strategy Ordering
        if self.meta_learner:
            self.strategies = self.meta_learner.get_dynamic_order(task_tier, self.ideology)
        
        # 3. Meta-Awareness Check
        if self.ideology == 'alpha' and self.meta_learner and task.task_id in self.meta_learner.global_solved_task_ids:
            return [copy.deepcopy(t) for t in task.test_inputs] 

        predictions = []
        for test_input in task.test_inputs:
            predictions.append(self._solve_single(test_input, task.train_examples, task.task_id, task_tier))
        return predictions
    
    def _solve_single(self, test_input: Grid, train_examples: List[Tuple[Grid, Grid]], task_id: str, task_tier: TaskTier) -> Grid:
        self.stats['total'] += 1
        
        for strategy in self.strategies:
            
            # The core: Try the rule, check if it works on ALL train examples.
            try:
                success, rule_params = strategy.can_apply(train_examples)
            except Exception:
                success, rule_params = False, None
                
            if success:
                result = strategy.apply(test_input, rule_params)
                
                # Check for successful transformation
                if result is not None and not GridOps.grids_equal(result, test_input):
                    self.stats['solved'] += 1
                    self.stats['strategy_success'][strategy.name] += 1
                    
                    if self.meta_learner:
                        self.meta_learner.update_metrics(strategy.name, task_tier, True)
                    
                    return result
            
            if self.meta_learner:
                self.meta_learner.update_metrics(strategy.name, task_tier, False)

        # Final Fallback
        return test_input

# =============================================================================
# --- DATA LOADING & EXECUTION LOGIC --- (x10 Segments)
# =============================================================================

class ARCDataLoader:
    @staticmethod
    def load_tasks(challenges_path: str) -> List[ARCTask]:
        try:
            with open(challenges_path, 'r') as f: challenges = json.load(f)
        except FileNotFoundError: 
            # In a competitive environment, this is critical, so we log
            print(f"WARNING: File not found at {challenges_path}. Returning empty list.")
            return []
        
        tasks = []
        for task_id, task_data in challenges.items():
            train_examples = [(ex['input'], ex['output']) for ex in task_data['train']]
            test_inputs = [ex['input'] for ex in task_data['test']]
            tasks.append(ARCTask(task_id=task_id, train_examples=train_examples, test_inputs=test_inputs))
        return tasks

class SubmissionGenerator:
    @staticmethod
    def generate(predictions: Dict[str, List[Grid]], output_path: str):
        submission = {}
        for task_id, pred_grids in predictions.items():
            # Only submit the first prediction attempt
            first_pred = pred_grids[0] if pred_grids and pred_grids[0] and isinstance(pred_grids[0], list) else []
            submission[task_id] = {
                "attempt_1": first_pred,
                "attempt_2": first_pred # Duplicate attempt_1 for safety
            }
        
        if output_path:
            with open(output_path, 'w') as f: json.dump(submission, f, indent=2)
            print(f"   Saved consolidated submission to: {output_path}")

def merge_predictions(current_predictions: Dict, new_predictions: Dict, meta_learner: MetaLearner) -> Dict:
    """Merges predictions and updates the MetaLearner's solved list."""
    newly_solved_ids = set()
    for task_id, new_preds in new_predictions.items():
        if not new_preds: continue
        new_grid = new_preds[0]
        
        # Check if new prediction is a solution (i.e., not a fallback)
        is_new_solution = new_grid and not GridOps.grids_equal(new_grid, [[-1]]) # Simple check for non-fallback

        if task_id not in current_predictions:
            current_predictions[task_id] = new_preds
            if is_new_solution: newly_solved_ids.add(task_id) 
        else:
            current_grid = current_predictions[task_id][0]
            # Overwrite if the new prediction is a solution and the current one is still a fallback
            if is_new_solution and not current_grid:
                 current_predictions[task_id] = new_preds
                 newly_solved_ids.add(task_id)
            
    meta_learner.global_solved_task_ids.update(newly_solved_ids)
    return current_predictions

def run_prediction_phase(
    solver: EnhancedARCSolver, 
    tasks: List[ARCTask], 
    time_limit_minutes: int, 
    phase_name: str,
    reverse_order: bool = False
):
    time_limit_seconds = time_limit_minutes * 60
    order_label = "REVERSED" if reverse_order else "NORMAL"
    print(f"\n--- {phase_name.upper()} | AGENT {solver.ideology.upper()} | {order_label} ORDER ({time_limit_minutes} MIN LIMIT) ---")
    
    solver.stats = {'total': 0, 'solved': 0, 'strategy_success': defaultdict(int)}
    
    tasks_to_run = tasks[::-1] if reverse_order else tasks
    
    start_time = time.time()
    predictions = {}
    tasks_attempted = 0
    
    for i, task in enumerate(tasks_to_run):
        elapsed_time = time.time() - start_time
        
        if elapsed_time >= time_limit_seconds:
            print(f"   Time limit reached! Stopping at task {i}/{len(tasks_to_run)}.")
            break
            
        if i > 0 and i % 20 == 0:
            tasks_per_second = i / elapsed_time if elapsed_time > 0 else 0
            # Keep logging progress, which will now show slower progression due to time sinks
            print(f"   Progress: {i}/{len(tasks_to_run)} | Elapsed: {elapsed_time:.0f}s | Speed: {tasks_per_second:.2f} tasks/sec")
            
        predictions[task.task_id] = solver.solve_task(task)
        tasks_attempted += 1

    final_elapsed = time.time() - start_time
    
    print(f"✅ {phase_name.upper()} COMPLETE!")
    print(f"   Tasks Attempted: {tasks_attempted} / {len(tasks_to_run)}")
    print(f"   Total Solutions Found: {solver.stats['solved']} / {solver.stats['total']} attempts")
    print(f"   Final Elapsed Time: {final_elapsed:.2f}s")
    print("-" * 80)
    
    return predictions


# =============================================================================
# --- MAIN EXECUTION (MCAS TRAINING & DEPLOYMENT LOOP) --- (x10 Segments)
# =============================================================================

def main():
    print("="*80)
    print("ARC PRIZE 2025 - ORCASWORD SOLVER v7.1 (META-COGNITIVE ADAPTIVE SYSTEM)")
    print("="*80)
    
    KAGGLE_INPUT_DIR = '/kaggle/input/arc-prize-2025'
    KAGGLE_OUTPUT_DIR = '/kaggle/working'
    
    TRAIN_PATH = f'{KAGGLE_INPUT_DIR}/arc-agi_training_challenges.json'
    TEST_PATH = f'{KAGGLE_INPUT_DIR}/arc-agi_test_challenges.json'          
    EVAL_PATH = f'{KAGGLE_INPUT_DIR}/arc-agi_evaluation_challenges.json'  
    
    TEST_OUTPUT_PATH = f'{KAGGLE_OUTPUT_DIR}/submission_test.json'
    EVAL_OUTPUT_PATH = f'{KAGGLE_OUTPUT_DIR}/submission_eval.json'
    
    print("\n📚 Loading data...")
    train_tasks = ARCDataLoader.load_tasks(TRAIN_PATH)
    test_tasks = ARCDataLoader.load_tasks(TEST_PATH)
    eval_tasks = ARCDataLoader.load_tasks(EVAL_PATH)
    print(f"   Loaded Train: {len(train_tasks)} | Test: {len(test_tasks)} | Eval: {len(eval_tasks)} tasks.")

    if not test_tasks and not eval_tasks: return

    # 1. Initialize MetaLearner
    initial_alpha_agent = EnhancedARCSolver(ideology='alpha')
    meta_learner = MetaLearner(initial_alpha_agent._get_all_strategies())

    # --- PHASE 0: META-TRAINING (Self-Validation & Learning) ---
    print("\n\n--- PHASE 0: META-TRAINING & SELF-VALIDATION (10 MIN) ---")
    
    meta_train_agent = EnhancedARCSolver(ideology='alpha', meta_learner=meta_learner)
    
    train_preds_p0 = run_prediction_phase(
        solver=meta_train_agent, tasks=train_tasks, time_limit_minutes=10, 
        phase_name="TRAINING CHALLENGES (P0 - Learn)", reverse_order=False
    )
    
    print("\n🧠 MetaLearner Knowledge Snapshot (Top Tier Success Rates):")
    for s_name, metrics in meta_learner.metrics.items():
        if metrics.total_attempts_by_tier['Elite'] > 0:
             rate = meta_learner.get_success_rate(s_name, 'Elite')
             print(f"   {s_name:<30} Elite Rate: {rate:.4f} ({metrics.success_count_by_tier['Elite']}/{metrics.total_attempts_by_tier['Elite']})")

    # --- PHASE 1 & 2: DYNAMIC DEPLOYMENT ---

    final_test_predictions = {}
    final_eval_predictions = {}
    
    alpha_agent = EnhancedARCSolver(ideology='alpha', meta_learner=meta_learner)
    omega_agent = EnhancedARCSolver(ideology='omega', meta_learner=meta_learner)

    # PHASE 1: Initial Coverage
    print("\n\n--- PHASE 1: DYNAMIC DEPLOYMENT (INITIAL COVERAGE) ---")
    
    test_preds_run1 = run_prediction_phase(
        solver=alpha_agent, tasks=test_tasks, time_limit_minutes=20, phase_name="TEST CHALLENGES (P1 - Alpha)", reverse_order=False
    )
    final_test_predictions = merge_predictions(final_test_predictions, test_preds_run1, meta_learner)
    
    eval_preds_run2 = run_prediction_phase(
        solver=omega_agent, tasks=eval_tasks, time_limit_minutes=30, phase_name="EVAL CHALLENGES (P1 - Omega)", reverse_order=True
    )
    final_eval_predictions = merge_predictions(final_eval_predictions, eval_preds_run2, meta_learner)


    # PHASE 2: Breakthrough & Consolidation 
    print("\n\n--- PHASE 2: DYNAMIC BREAKTHROUGH (ROLES SWAPPED) ---")

    eval_preds_run3 = run_prediction_phase(
        solver=alpha_agent, tasks=eval_tasks, time_limit_minutes=30, phase_name="EVAL CHALLENGES (P2 - Alpha)", reverse_order=False
    )
    final_eval_predictions = merge_predictions(final_eval_predictions, eval_preds_run3, meta_learner)

    test_preds_run4 = run_prediction_phase(
        solver=omega_agent, tasks=test_tasks, time_limit_minutes=20, phase_name="TEST CHALLENGES (P2 - Omega)", reverse_order=True
    )
    final_test_predictions = merge_predictions(final_test_predictions, test_preds_run4, meta_learner)


    # --- FINAL SUBMISSION GENERATION ---
    print("\n" + "="*80)
    print("🔥 AGGREGATING AND GENERATING FINAL SUBMISSIONS")
    
    SubmissionGenerator.generate(final_test_predictions, TEST_OUTPUT_PATH)
    SubmissionGenerator.generate(final_eval_predictions, EVAL_OUTPUT_PATH)
    
    print("\n✅ Execution Complete. Final submissions written.")
    print(f"Total Unique Test Tasks Solved (Consolidated): {len(meta_learner.global_solved_task_ids.intersection(final_test_predictions.keys()))}")
    print(f"Total Unique Eval Tasks Solved (Consolidated): {len(meta_learner.global_solved_task_ids.intersection(final_eval_predictions.keys()))}")
    print("="*80)

# Explicitly call main() for robust execution
import numpy as np 
main()
#Cell 1


ARC PRIZE 2025 - ORCASWORD SOLVER v7.1 (META-COGNITIVE ADAPTIVE SYSTEM)

📚 Loading data...
   Loaded Train: 1000 | Test: 240 | Eval: 120 tasks.
   MetaLearner initialized with 14 strategies.


--- PHASE 0: META-TRAINING & SELF-VALIDATION (10 MIN) ---

--- TRAINING CHALLENGES (P0 - LEARN) | AGENT ALPHA | NORMAL ORDER (10 MIN LIMIT) ---
   Progress: 20/1000 | Elapsed: 2s | Speed: 11.17 tasks/sec
   Progress: 40/1000 | Elapsed: 4s | Speed: 10.84 tasks/sec
   Progress: 60/1000 | Elapsed: 6s | Speed: 10.68 tasks/sec
   Progress: 80/1000 | Elapsed: 8s | Speed: 10.09 tasks/sec
   Progress: 100/1000 | Elapsed: 10s | Speed: 9.64 tasks/sec
   Progress: 120/1000 | Elapsed: 12s | Speed: 9.67 tasks/sec
   Progress: 140/1000 | Elapsed: 15s | Speed: 9.44 tasks/sec
   Progress: 160/1000 | Elapsed: 16s | Speed: 9.72 tasks/sec
   Progress: 180/1000 | Elapsed: 19s | Speed: 9.37 tasks/sec
   Progress: 200/1000 | Elapsed: 21s | Speed: 9.31 tasks/sec
   Progress: 220/1000 | Elapsed: 23s | Speed: 9.42 tasks/